In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os, time, random
from bs4 import BeautifulSoup
import urllib.request


def chromeWebdriver():
    options = Options()
    options.add_argument("lang=ko_KR")  # 언어 설정
    # options.add_argument("start-maximized") # 창 크기 최대로
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")    
    options.add_experimental_option('detach', True)  # 브라우저 안 닫히게
    options.add_experimental_option('excludeSwitches', ['enable-logging'])  # 시스템 장치 에러 숨기기
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')    
    # options.add_argument('--headless')  # 웹 브라우저를 시각적으로 띄우지 않는 headless chrome 옵션
    driver = webdriver.Chrome(service=Service(executable_path=ChromeDriverManager().install()), options=options)
    return driver


def collect_image(search_word, extract_img_count):
    url = 'https://www.google.co.kr'

    now = time.localtime()
    today_time = f'{now.tm_year}{now.tm_mon}{now.tm_mday}_{now.tm_hour}{now.tm_min}'
    print(today_time)

    file_path = "C:/Users/wodms/Jupyter_python/google_crawling/"

    os.chdir(file_path)
    os.makedirs(file_path + today_time + '_' + search_word)
    os.chdir(file_path + today_time + '_' + search_word)
    file_save_dir = file_path + today_time + '_' + search_word
    print(file_save_dir)

    driver = chromeWebdriver()
    driver.get(url)
    time.sleep(random.uniform(2, 3))
    elem_q = driver.find_element(By.NAME, 'q')
    elem_q.send_keys(search_word)
    elem_q.submit()

    driver.find_element(By.LINK_TEXT, '이미지').click()  # 텍스트 메뉴 '이미지' 링크 클릭
    # driver.find_element(By.XPATH, '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a').click()
    time.sleep(random.uniform(1, 2))

    # 페이지 스크롤 다운
    def page_scrolling(drivers):
        ## scrolling ------------------------------
        elem = driver.find_element(By.TAG_NAME, 'body')
        page_height = driver.execute_script('return document.body.scrollHeight')
        # print(page_height)

        # more_view_cnt = 0
        scroll_cnt = 1
        more_view_scroll_cnt = -1  # '결과 더보기' 버튼 나올 때의 scroll_cnt (break 처리 위해 사용)
        equal_cnt = 1
        while True:
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(random.uniform(0.3, 0.5))
            new_height = driver.execute_script('return document.body.scrollHeight')
            if page_height != new_height:
                page_height = new_height
                equal_cnt = 1
            print(f'scroll_cnt: {scroll_cnt}, new_height: {new_height}, equal_cnt: {equal_cnt}')
            
            try:
                scroll_cnt += 1
                equal_cnt += 1
                driver.find_element(By.XPATH, '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input').click()  # 결과 더보기 버튼 처리
                print('결과 더보기 버튼 클릭 처리')
                more_view_scroll_cnt = scroll_cnt
                more_view_cnt += 1
            except:
                if equal_cnt == 20:  # scroll_cnt / more_view_scroll_cnt > 2.5:
                    break
                continue
        ## End of scrolling ------------------------------

    page_scrolling(driver)

    file_no = 1
    count = 1
    img_src = []

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # print(soup)
    # imgs = driver.find_elements(By.TAG_NAME, 'img')
    imgs = driver.find_elements(By.CSS_SELECTOR, '#islrg > div.islrc > div a.wXeWr.islib.nfEiy')
    print(len(imgs))

    for img in imgs:
        img_src1 = img.click()  # 이미지 클릭 시 display 되는 url을 찾기 위해 클릭함
        try:
            img_src2 = driver.find_element(By.CSS_SELECTOR, '#Sva75c > div > div > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id > div > a')
        except Exception:
            continue
        time.sleep(random.uniform(0.2, 0.5))
        img_src3 = img_src2.find_element(By.TAG_NAME, 'img').get_attribute('src')
        if img_src3[:4] != 'http':
            continue
        print(count, img_src3, '\n')

        img_src.append(img_src3)
        if count == extract_img_count + 10:  # 이미지 에러 대비해서 입력 숫자보다 크게 잡음
            break        
        count += 1
        
    print(f'\n{"="*10} 추출한 전체 리스트 {"="*10}\n{img_src}\n\n{"="*10}총 {len(img_src)}개 추출함{"="*10}\n')

    for i in range(len(img_src)):
        extention = img_src[i].split('.')[-1]
        ext = ''
        if extention in ('jpg', 'JPG', 'jpeg', 'JPEG', 'png', 'PNG', 'gif', 'GIF'):
            ext = '.' + extention
        else:
            ext = '.jpg'        
        try:
            urllib.request.urlretrieve(img_src[i], '자바 칩 프라푸치노' + str(file_no).zfill(3) + ext)
            print(img_src[i])
        except Exception:
            continue

        print(f'{file_no}번째 이미지 저장-----')
        file_no += 1
        
        if file_no - 1 == extract_img_count:
            break

    driver.close()


if __name__ == '__main__':
    collect_image('자바 칩 프라푸치노', 400)

2022817_2320
C:/Users/wodms/Jupyter_python/google_crawling/2022817_2320_자바 칩 프라푸치노

scroll_cnt: 1, new_height: 3028, equal_cnt: 1
scroll_cnt: 2, new_height: 3028, equal_cnt: 2
scroll_cnt: 3, new_height: 5702, equal_cnt: 1
scroll_cnt: 4, new_height: 5702, equal_cnt: 2
scroll_cnt: 5, new_height: 5702, equal_cnt: 3
scroll_cnt: 6, new_height: 5702, equal_cnt: 4
scroll_cnt: 7, new_height: 5702, equal_cnt: 5
scroll_cnt: 8, new_height: 5702, equal_cnt: 6
scroll_cnt: 9, new_height: 11526, equal_cnt: 1
scroll_cnt: 10, new_height: 11526, equal_cnt: 2
scroll_cnt: 11, new_height: 11526, equal_cnt: 3
scroll_cnt: 12, new_height: 11526, equal_cnt: 4
scroll_cnt: 13, new_height: 11526, equal_cnt: 5
scroll_cnt: 14, new_height: 11526, equal_cnt: 6
scroll_cnt: 15, new_height: 11526, equal_cnt: 7
scroll_cnt: 16, new_height: 11526, equal_cnt: 8
scroll_cnt: 17, new_height: 11526, equal_cnt: 9
scroll_cnt: 18, new_height: 11526, equal_cnt: 10
scroll_cnt: 19, new_height: 17108, equal_cnt: 1
scroll_cnt: 20, new_

48 https://ssproxy.ucloudbiz.olleh.com/v1/AUTH_e59809eb-bdc9-44d7-9d8f-2e7f0e47ba91/post_card/38996_1534917446_fjhPvyqd.JPG 

49 https://img.danawa.com/images/descFiles/4/963/3962982_1563193931508.jpeg 

50 https://pbs.twimg.com/media/CrPbykEUEAELC7I.jpg:large 

51 https://media.bunjang.co.kr/product/194912478_1_1659112747_w360.jpg 

52 https://live.staticflickr.com/2266/5721129977_5860eefa46_z.jpg 

53 https://t1.daumcdn.net/cfile/tistory/2325474B550834650C 

54 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcShUgx-Jo84pf8R925uq_JfkU7zHP_iWNneEw&usqp=CAU 

55 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ6CmKUbidADsKxf3L37iQMGE1goM9A43VyOQ&usqp=CAU 

56 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTdfT6ULg1d3kpmFHO4F53bh-HuaG3GA9ZXIA&usqp=CAU 

57 http://casebuycase2.cafe24.com/web/upload/NNEditor/20151217/IMG_6925.jpg 

58 https://t1.daumcdn.net/cfile/tistory/2373704158E8F08224 

59 https://t1.daumcdn.net/cfile/tistory/990F27355B9BD3722A 

60 http://images.coo

137 https://cdn.crowdpic.net/detail-thumb/thumb_d_8D867F47FB60D3EB7A2BC954DBFFFB96.jpg 

138 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTj0-FNgFRJEdgEt-ELsk2pmPckhGDJDffdIA&usqp=CAU 

139 http://webimage.10x10.co.kr/item/contentsimage/244/F002447296_03.jpg 

140 https://cdn.wadiz.kr/ft/images/green002/2018/0515/20180515164428970_66.jpg/wadiz/thumbnail/1081.5/quality/60/optimize/ 

141 https://t1.daumcdn.net/cfile/tistory/1758D8494D173BA824?original 

142 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSiUQLSGuCIEeOza4MNAzCRJvQB_Lb-AdCvng&usqp=CAU 

143 https://img.hankyung.com/photo/202001/AA.21615668.1-1200x.jpg 

144 https://contents.lotteon.com/itemimage/LO/17/88/08/32/20/_1/78/80/83/22/1/LO1788083220_1788083221_1.jpg/dims/optimize/dims/resizemc/400x400 

145 https://todaycoffee.kr/data/file/proofshot/thumb-3416707498_hna92Fre_48db7810935166cd254b03c1726c42537560b9bb_980x1307.jpg 

146 https://t1.daumcdn.net/cfile/tistory/240DB24C5325219D25 

147 http://www.istarbuc

216 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR895BzUJe0zzSXswLxktkk3Kx82DVPAPD0WA&usqp=CAU 

217 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRC59x5E6FBZqZu4veel6rV2gZV0cEcSFoTtQ&usqp=CAU 

218 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR9Bfdtkm5_M0imsMFgC4BxFI9uW8AuJZlqyg&usqp=CAU 

219 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR3EAZMjHdYtGORC52rj2klU_GCCkexmWgxWg&usqp=CAU 

220 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQD-zThfHG4wmYL6CASPjlTtzlGSLUKt3Z8Kw&usqp=CAU 

221 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwvnSqzcn5ObKqXPJzkundJLCdAPPPMQ-NBw&usqp=CAU 

222 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQewPAAABPLvszwCYJRo6ufHnkCo4PUnoBhRA&usqp=CAU 

223 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRe6yAf0HijPC5hGRxK9GtqxiB9yKog-uuvRQ&usqp=CAU 

224 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTcVol1qGER6YUbcVNYdq6g95zMyFUELi3E7g&usqp=CAU 

225 https://encrypted-tbn0.gstatic.com/images?

294 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmPoiCqxNA9jw6jTnza8gjnRlvZMdFbJz_AA&usqp=CAU 

295 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQKRZ_In6Q-AYrVR9qKs34pmr-Tuqb9dkvTZg&usqp=CAU 

296 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSKSYrFKdENsE6lQOHDPJc7oVWlOuQLOAp43Q&usqp=CAU 

297 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ42mdh87ZpWn2b4vvngyxTvbGGbUcZXcWang&usqp=CAU 

298 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRIxXE5u4fZFPOPaPvFmu6hH3KS-3fKq1jo0A&usqp=CAU 

299 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSBW6PkiL0G7T1oE8tcHLIIUIjaK87ieq9NaQ&usqp=CAU 

300 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ7uS7P4-4B6szK1D0u5_-Wq1OE--ys-gGNeA&usqp=CAU 

301 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTPZjsCZvXYqhVNQGNGHWS5O0xaQKHJc_tLDw&usqp=CAU 

302 https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2k1D4LWg56AfhG6aaB0vsRpYDX1zSY38j6g&usqp=CAU 

303 https://encrypted-tbn0.gstatic.com/images?

...

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTCPifa32butX8uUT9Hv8IVnI5nOJQ1Hrx9sA&usqp=CAU
81번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4-_ze9Kvm9oRlOguAaXmXT2kyXMwhUqFN0Q&usqp=CAU
82번째 이미지 저장-----
https://t1.daumcdn.net/cfile/tistory/9990743C5CD96D1813
83번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSAIiqucAQMRn_H3rvpoUZq176EBirOOqORmg&usqp=CAU
84번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJGH5VA5sPnNKELXQYAzj9bDEXJnT6MGMW2Q&usqp=CAU
85번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWNhZe8J4t2Pm_bYzNWQQ5H6QZQK2YiL2k2Q&usqp=CAU
86번째 이미지 저장-----
https://res.heraldm.com/phpwas/restmb_idxmake.php?idx=621&simg=/content/image/2015/01/29/20150129000486_0.jpg
87번째 이미지 저장-----
https://semosang.co.kr/data/goods/1/2020/etc1/25-300.jpg
88번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTlGWkTm6gKI1I9iGK5DYtVdAPo1Dn5gvSX4Q&usqp=CAU
89번째 이미지 저장-----
https://newsimg.sedaily.

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTH-btPb2yhSwVHPtAUMoWMTso-H8OiAPhBVQ&usqp=CAU
156번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0-rMfXVAMVMr9ORJBKyw8gMxUpwHtYsaLaA&usqp=CAU
157번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRKgJs9sjsy9BeiHHbcAt_Z8aoh2oGgiTHZAA&usqp=CAU
158번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRHvqyy8FEbaepIcU3SJKxpZlXopr7m0u_vmw&usqp=CAU
159번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQMFWu8KpYdydHqhw_zyHiSGWwGt9xKjogDvw&usqp=CAU
160번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTSKk7CyFcKtO7y4Sh-QqexfkwaeWAdTcrGJA&usqp=CAU
161번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTdFczR4j05vFUvor-lz6B10qcCdlPPdfoWBA&usqp=CAU
162번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcScad06_AWF94v6-krPjylMjidYp8T-usLhog&usqp=CAU
163번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ2

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRwNXiFf36lxEhIurk1FTZmNYbSR2CtqP9b6g&usqp=CAU
226번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRQ3xkYqIVCVSDbu3Qd-cF_01VF4M_Azgn2FQ&usqp=CAU
227번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcStEPYCtJgIxJHfBJgBgRMsazy5LNpPGwM7XA&usqp=CAU
228번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQMIXfn34EfAb2iNGyfgfPTA0CYE0Yz5Vtl6A&usqp=CAU
229번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRhZdEGf9NmkmcIQTrTZDfUw7_eNZYVTZ-51g&usqp=CAU
230번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRLmR0ZEgU2wGEBLM6LwrYMJJ-PMCdcnYzE4A&usqp=CAU
231번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRiqe4g5FTP2VTOl4YJ8qFzX4Th-TGdPnaVhA&usqp=CAU
232번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTo8tkBSOk6HfTOaUCxAL4kF0v4hD6ahhL5tw&usqp=CAU
233번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSN

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSsSfoPkC509EW2toASSw8jPGRK99w_Cr4DKA&usqp=CAU
296번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRyUwMRVdnaZA0esPSnxqYsTK4TWn8-0VMedg&usqp=CAU
297번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQHbbwcQ0OdwHNNMqazwkX1f2cgc40F5RFhHQ&usqp=CAU
298번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTXL2Gmk93W2gw-Zyha_eoplPsUOZRaytygTg&usqp=CAU
299번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPJzs1nniHpXXiu_PsaIbYI-NlEOlZOqkGjg&usqp=CAU
300번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQNKwVrQPKKBQG4C5g0difOIU-uPKi7wltKng&usqp=CAU
301번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcToNYgKwzr5Im5vWbnmgBKBG4mwX-r4d6WuYg&usqp=CAU
302번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwY-3PD8RsZkYAN0hgaueSnu93CTr6K_RJYw&usqp=CAU
303번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQB

https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS2qtgDodkgfhTRU42IdVV6GIYQAsJHkDbkDg&usqp=CAU
366번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ8vmKthJCkIKS-JSbTYD7vdV1ea-z1PDra4w&usqp=CAU
367번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQels9PFF86QW5UQWOD3xHBy50GBo1v3hyAow&usqp=CAU
368번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQH0pK2qPl_C2evdJsG_VG6-XIrn0wEkwgSKA&usqp=CAU
369번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR5tkAw0VE_6uGpj0EhWnTptC4vdlYFhjlNHw&usqp=CAU
370번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQC_MkrrKa7kMYIZjLp0iMyxhjPpY4Y4D6AGA&usqp=CAU
371번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ4OFVhpzVaflpW-7Bq8Djp0nNsHJor7tZjlQ&usqp=CAU
372번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSLm3EvIioYG-b7CgyhZk41Z8mjaPFEYlKn2w&usqp=CAU
373번째 이미지 저장-----
https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRn